# Input/output with files

C++ provides the following classes to perform output and input of characters to/from files:

* `ofstream`: Stream class to write on files
* `ifstream`: Stream class to read from files
* `fstream`: Stream class to both read and write from/to files.

These classes are derived directly or indirectly from the classes `istream` and `ostream`. We have already used objects whose types were these classes: `cin` is an object of class `istream` and `cout` is an object of class `ostream`. Therefore, we have already been using classes that are related to our file streams. And in fact, we can use our file streams the same way we are already used to use cin and cout, with the only difference that we have to associate these streams with physical files.


## Open a file

The first operation generally performed on an object of one of these classes is to associate it to a real file. This procedure is known as to open a file. An open file is represented within a program by a stream and any input or output operation performed on this stream object will be applied to the physical file associated with it.

To open a file with a stream object we use its member function open:

```c++
open (filename, mode);
```

Where filename is a string representing the name of the file to be opened, and mode is an optional parameter with a combination of the following flags:

<p align="center">
    <table class="boxed">
<tbody><tr><td><code>ios::in</code></td><td>Open for input operations.</td></tr>
<tr><td><code>ios::out</code></td><td>Open for output operations.</td></tr>
<tr><td><code>ios::binary</code></td><td>Open in binary mode.</td></tr>
<tr><td><code>ios::ate</code></td><td>Set the initial position at the end of the file.<br>
If this flag is not set, the initial position is the beginning of the file.</td></tr>
<tr><td><code>ios::app</code></td><td>All output operations are performed at the end of the file, appending the content to the current content of the file.</td></tr>
<tr><td><code>ios::trunc</code></td><td>If the file is opened for output operations and it already exists, its previous content is deleted and replaced by the new one.</td></tr>
</tbody></table>
</p>

All these flags can be combined using the bitwise operator OR (|). For example, if we want to open the file example.bin in binary mode to add data we could do it by the following call to the member function open:

```c++
ofstream myfile;
myfile.open ("example.bin", ios::out | ios::app | ios::binary);
```

Each of the open member functions of classes `ofstream`, `ifstream`, and `fstream` has a default mode that is used if the file is opened without a second argument:

<p align="center">
    <table class="boxed">
<tbody><tr><th>class</th><th>default mode parameter</th></tr>
<tr><td><code>ofstream</code></td><td>ios::out</td></tr>
<tr><td><code>ifstream</code></td><td>ios::in</td></tr>
<tr><td><code>fstream</code></td><td>ios::in | ios::out</td></tr>
</tbody></table>
</p>

For `ifstream` and `ofstream` classes, `ios::in` and `ios::out` are automatically and respectively assumed, even if a mode that does not include them is passed as the second argument to the open member function (the flags are combined).

For `fstream`, the default value is only applied if the function is called without specifying any value for the mode parameter. If the function is called with any value in that parameter the default mode is overridden, not combined.

File streams opened in binary mode perform input and output operations independently of any format considerations. Non-binary files are known as text files, and some translations may occur due to the formatting of some special characters (like newline and carriage return characters).

Since the first task that is performed on a file stream is generally to open a file, these three classes include a constructor that automatically calls the open member function and has the same parameters as this member. For example, we could also have declared the previous object and conducted the same opening operation in our previous example by writing:

```c++
ofstream myfile ("example.bin", ios::out | ios::app | ios::binary);
```

Combining object construction and stream opening in a single statement. Both forms to open a file are valid and equivalent.

To check if a file stream was successful opening a file, you can do it by calling to member is_open. This member function returns a bool value of true in the case that indeed the stream object is associated with an open file, or false otherwise:

```c++
if (myfile.is_open()) { /* ok, proceed with output */ }
```


## Closing a file

When we are finished with our input and output operations on a file we shall close it so that the operating system is notified and its resources become available again. For that, we call the stream's member function close. This member function takes flushes the associated buffers and closes the file:

```c++
myfile.close();
```

Once this member function is called, the stream object can be re-used to open another file, and the file is available again to be opened by other processes.

In case that an object is destroyed while still associated with an open file, the destructor automatically calls the member function `close`.


## Text files

Text file streams are those where the `ios::binary` flag is not included in their opening mode. These files are designed to store text and thus all values that are input or output from/to them can suffer some formatting transformations, which do not necessarily correspond to their literal binary value.

Writing operations on text files are performed in the same way we operated with `cout`:


In [ ]:
std::cout << "Enter your name: ";
std::string yourName;
std::cin >> yourName;

std::ofstream myfile("sample_io/input.txt", std::ios::app); // open for appending
if (myfile.is_open()) {
    myfile << "\nGood to see you, " << yourName << std::endl;
    myfile.close();

    // Reopen the file for reading
    std::ifstream readFile("sample_io/input.txt");
    std::string content;
    if (readFile.is_open()) {
        while (std::getline(readFile, content)) {
            std::cout << content << std::endl;
        }
        readFile.close();
    } else {
        std::cout << "Unable to open file for reading";
    }
} else {
    std::cout << "Unable to open file for writing";
}

## Checking state flags

The following member functions exist to check for specific states of a stream (all of them return a `bool` value):

* `bad()`: Returns true if a reading or writing operation fails. For example, in the case that we try to write to a file that is not open for writing or if the device where we try to write has no space left.
* `fail()`: Returns true in the same cases as bad(), but also in the case that a format error happens, like when an alphabetical character is extracted when we are trying to read an integer number.
* `eof()`: Returns true if a file open for reading has reached the end.
* `good()`: It is the most generic state flag: it returns false in the same cases in which calling any of the previous functions would return true. Note that good and bad are not exact opposites (good checks more state flags at once).

The member function `clear()` can be used to reset the state flags.


## Checking state flags

All I/O stream objects keep internally -at least- one internal position:

* `ifstream`, like `istream`, keeps an internal get position with the location of the element to be read in the next input operation.
* `ofstream`, like `ostream`, keeps an internal put position with the location where the next element has to be written.

Finally, fstream, keeps both, the get and the put position, like `iostream`.

These internal stream positions point to the locations within the stream where the next reading or writing operation is performed. These positions can be observed and modified using the following member functions:

* `tellg()` and `tellp()`: These two member functions with no parameters return a value of the member type `streampos`, which is a type representing the current get position (in the case of tellg) or the put position (in the case of `tellp`).

* `seekg()` and `seekp()`: These functions allow to change the location of the get and put positions. Both functions are overloaded with two different prototypes. The first form is:

```c++
seekg ( position );
seekp ( position );
```

Using this prototype, the stream pointer is changed to the absolute position position (counting from the beginning of the file). The type for this parameter is streampos, which is the same type as returned by functions `tellg` and `tellp`.

The other form for these functions is:

```c++
seekg ( offset, direction );
seekp ( offset, direction );
```

Using this prototype, the **get or put position** is set to an offset value relative to some specific point determined by the parameter direction. `offset` is of type `streamoff`. And the direction is of type `seekdir`, which is an enumerated type that determines the point from where `offset` is counted, and that can take any of the following values:

<p align="center">
    <table class="boxed">
<tbody><tr><td><code>ios::beg</code></td><td>offset counted from the beginning of the stream</td></tr>
<tr><td><code>ios::cur</code></td><td>offset counted from the current position</td></tr>
<tr><td><code>ios::end</code></td><td>offset counted from the end of the stream</td></tr>
</tbody></table>
</p>

The following example uses the member functions we have just seen to obtain the size of a file. Notice the `streampos` we have used in the example. 

`streampos` is a specific type used for buffer and file positioning and is the type returned by `file.tellg()`. Values of this type can safely be subtracted from other values of the same type, and can also be converted to an integer type large enough to contain the size of the file. These stream positioning functions use two particular types: `streampos` and `streamoff`. 

These types are also defined as member types of the stream class. Each of the member types above is an alias of its non-member equivalent (they are the same type). It does not matter which one is used. The member types are more generic because they are the same on all stream objects (even on streams using exotic types of characters), but the non-member types are widely used in existing code for historical reasons:

<p align="center">
<table class="boxed">
<tbody><tr><th>Type</th><th>Member type</th><th>Description</th></tr>
<tr><td><code><a href="/streampos">streampos</a></code></td><td><code><a href="/ios#types">ios::pos_type</a></code></td><td>Defined as <code><a href="/fpos">fpos&lt;mbstate_t&gt;</a></code>.<br>
It can be converted to/from <code><a href="/streamoff">streamoff</a></code> and can be added or subtracted values of these types.</td></tr>
<tr><td><code><a href="/streamoff">streamoff</a></code></td><td><code><a href="/ios#types">ios::off_type</a></code></td><td>It is an alias of one of the fundamental integral types (such as <code>int</code> or <code>long long</code>).</td></tr>
</tbody></table>
</p>


In [ ]:
std::streampos begin,end;
std::ifstream myfile ("sample_io/input.txt", std::ios::app);
begin = myfile.tellg();
myfile.seekg (0, std::ios::end);
end = myfile.tellg();
myfile.close();
std::cout << "size is: " << (end-begin) << " bytes.\n";

## Binary files

For binary files, reading and writing data with the extraction and insertion operators (`<<` and `>>`) and functions like getline is not efficient, since we do not need to format any data and data is likely not formatted in lines.

File streams include two member functions specifically designed to read and write binary data sequentially: write and read. The first one (write) is a member function of ostream (inherited by `ofstream`). And read is a member function of `istream` (inherited by `ifstream`). Objects of class `fstream` have both. Their prototypes are:

```c++
write ( memory_block, size );
read ( memory_block, size );
```

Where memory_block is of type `char*` (pointer to `char`), and represents the address of an array of bytes where the read data elements are stored or from where the data elements to be written are taken. The size parameter is an integer value that specifies the number of characters to be read or written from/to the memory block.


In [ ]:
std::streampos size;
char* memblock;

std::ifstream file ("sample_io/input.txt", std::ios::in|std::ios::binary|std::ios::ate);

if (file.is_open())
{
    size = file.tellg();
    memblock = new char [size];
    file.seekg (0, std::ios::beg);
    file.read (memblock, size);
    file.close();
    
    std::cout << "the entire file content is in memory";
    
    delete[] memblock;
} else { 
    std::cout << "Unable to open file"; 
}

## Buffers and Synchronization

When we operate with file streams, these are associated to an internal buffer object of type `streambuf`. This buffer object may represent a memory block that acts as an intermediary between the stream and the physical file. For example, with an `ofstream`, each time the member function `put` (which writes a single character) is called, the character may be inserted in this intermediate buffer instead of being written directly to the physical file with which the stream is associated.

The operating system may also define other layers of buffering for reading and writing to files.

When the buffer is flushed, all the data contained in it is written to the physical medium (if it is an output stream). This process is called synchronization and takes place under any of the following circumstances:

* **When the file is closed**: before closing a file, all buffers that have not yet been flushed are synchronized and all pending data is written or read to the physical medium.
* **When the buffer is full**: Buffers have a certain size. When the buffer is full it is automatically synchronized.
* **Explicitly, with manipulators**: When certain manipulators are used on streams, an explicit synchronization takes place. These manipulators are: flush and endl.
* **Explicitly, with member function `sync()`**: Calling the stream's member function sync() causes an immediate synchronization. This function returns an int value equal to -1 if the stream has no associated buffer or in case of failure. Otherwise (if the stream buffer was successfully synchronized) it returns 0.
